# Matrix multiplication with Pycuda

By Hugo Perrin & Eulalie Formery

In [0]:
!pip install pycuda
!apt-get update
!apt install -y --no-install-recommends -q python3-pycuda

    100% |████████████████████████████████| 1.6MB 15.2MB/s 
    100% |████████████████████████████████| 61kB 22.9MB/s 
    100% |████████████████████████████████| 573kB 26.6MB/s 
  Running setup.py bdist_wheel for pycuda ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/a5/17/ac/99922221c732eeece43529d3e0f9d441f7301c75990b2cdbff

Pour ce projet, nous avons utilisé Google Colaboratory, nous permettant d'installer pycuda facilement. 

In [0]:
import numpy as np
from pycuda import driver, compiler, gpuarray, tools
import pycuda.autoinit
import timeit
import matplotlib.pyplot as plt
%matplotlib inline
from timeit import default_timer as timer

## **Première version** : Parallélisation sur un seul block



Dans un GPU, chaque grid est composé de blocks ayant eux même plusieurs threads, sur lesquels nous pouvons paralléliser des calculs. La première fonction ne parallélise le produit matriciel qu'au sein d'un block. Le nombre de threads par blocks étant de 32 ici, avec cette fonction la dimension des matrices ne peut pas dépasser 32.

In [0]:
matmul_kernel = """
__global__ void MatMul(float *a, float *b, float *c)
{

int tx = threadIdx.x;
int ty = threadIdx.y;


 float value = 0;


for (int k = 0; k < %(DIM_MATRIX)s; ++k) {
  float Aelement = a[ty * %(DIM_MATRIX)s + k];
  float Belement = b[k * %(DIM_MATRIX)s + tx];
  value += Aelement * Belement;
 }

c[ty * %(DIM_MATRIX)s + tx] = value;
 }
"""

dim_matrix = 32

a_cpu = np.random.randn(dim_matrix, dim_matrix).astype(np.float32)
b_cpu = np.random.randn(dim_matrix, dim_matrix).astype(np.float32)

c_cpu = np.dot(a_cpu, b_cpu)

# Transfert des matrices sur le gpu
a_gpu = gpuarray.to_gpu(a_cpu) 
b_gpu = gpuarray.to_gpu(b_cpu)


# Création d'un array vide dans le gpu
c_gpu = gpuarray.empty((dim_matrix, dim_matrix), np.float32)

matmul_cuda= matmul_kernel % {
'DIM_MATRIX': dim_matrix
     }

mod = compiler.SourceModule(matmul_cuda)
matrixmul = mod.get_function("MatMul")
matrixmul(a_gpu, b_gpu, c_gpu, block = (dim_matrix, dim_matrix, 1),)

print(c_cpu - c_gpu.get())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## **Deuxième version**: Parallélisation sur plusieurs blocks

In [0]:
matmul_kernel_large = """
__global__ void MatMul(float *a, float *b, float *c) 
{

int TileWidth = blockDim.x;
int tx = (TileWidth*blockIdx.x) + threadIdx.x;
int ty = (TileWidth*blockIdx.y) + threadIdx.y;

float value = 0;

for (int k = 0; k < %(DIM_MATRIX)s; ++k) {

  float Aelement = a[ty * %(DIM_MATRIX)s + k];
  float Belement = b[k * %(DIM_MATRIX)s + tx];
  value += Aelement * Belement;
  
 }

c[ty * %(DIM_MATRIX)s + tx] = value;
}
"""


dim_matrix = 500
BLOCK_SIZE = 32

# Choix du gridsize
if dim_matrix%BLOCK_SIZE != 0:
    grid=(int(dim_matrix / BLOCK_SIZE + 1), int(dim_matrix / BLOCK_SIZE + 1),1)
else:
    grid=(1,1,1)
    
    
a_cpu = np.random.randn(dim_matrix, dim_matrix).astype(np.float32)
b_cpu = np.random.randn(dim_matrix, dim_matrix).astype(np.float32)

# Produit matriciel avec numpy : pour comparer le temps d'exécution
start = timer()

c_cpu = np.dot(a_cpu, b_cpu)

end = timer()

print('Time for a Matrix multiplication with numpy: %.5f seconds' %(end - start))

start_total = timer()

# Transfert des matrices sur le GPU
a_gpu = gpuarray.to_gpu(a_cpu) 
b_gpu = gpuarray.to_gpu(b_cpu)


# Création d'un array vide pour stocker le résultat
c_gpu = gpuarray.empty((dim_matrix, dim_matrix), np.float32)


matmul_cuda = matmul_kernel_large % {
'DIM_MATRIX': dim_matrix
     }

mod = compiler.SourceModule(matmul_cuda)
matrixmul = mod.get_function("MatMul")

start_calcul = timer()
 
matrixmul(a_gpu, b_gpu,c_gpu, block = (32, 32, 1), grid=grid)

end = timer()

print('Time for a matrix multiplication with pycuda: %.5f seconds' %(end - start_calcul))
print('Time for a matrix multiplication with pycuda and transfer to GPU: %.5f seconds' %(end - start_total))



Time for a Matrix multiplication with numpy: 0.00407 seconds
Time for a matrix multiplication with pycuda: 0.00031 seconds
Time for a matrix multiplication with pycuda and transfer to GPU: 0.34258 seconds
